After you have run the GetDataSDOT and you have data files containing transactions (e.g., 02142017_03232017.csv), then you can run this notebook to compute the loads.

In [1]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import os
import calendar
import glob
import datetime
import load_sdot_utils
import pandas as pd
from collections import defaultdict

In [2]:
curr_path = os.getcwd()
data_path = curr_path + '/../data'
raw_transaction_path = data_path + '/RawTransactionData'
belltown_path = data_path + '/Belltown_Minute'
commcore_path = data_path + '/CommercialCore_Minute'
pikepine_path = data_path + '/PikePine_Minute'
firsthill_path = data_path + '/FirstHill_Minute'
dennytriangle_path = data_path + '/DennyTriangle_Minute'

In [ ]:
pd.read_csv(os.path.join(data_path, 'block_info.csv'))['PaidParkingArea'].unique()

In [ ]:
# load_sdot_utils.get_data(3, 2016, file_path=raw_transaction_path, verbose=True)

In [8]:
filenames = []
for fi in glob.glob(raw_transaction_path + '/*.csv'):
    print fi.split('/')[-1]
    filenames.append(fi)

01012017_01312017.csv
02012017_02282017.csv
03012016_03312016.csv
03012017_03312017.csv
04012016_04302016.csv
04012017_04302017.csv
05012016_05312016.csv
05012017_05312017.csv
06012016_06302016.csv
06012017_06302017.csv
07012016_07312016.csv
07012017_07312017.csv


In [5]:
sorted(filenames)

['/Users/tfiez/GitHub/spatial-data-analysis/code/../data/RawTransactionData/01012017_01312017.csv',
 '/Users/tfiez/GitHub/spatial-data-analysis/code/../data/RawTransactionData/02012017_02282017.csv',
 '/Users/tfiez/GitHub/spatial-data-analysis/code/../data/RawTransactionData/03012016_03312016.csv',
 '/Users/tfiez/GitHub/spatial-data-analysis/code/../data/RawTransactionData/03012017_03312017.csv',
 '/Users/tfiez/GitHub/spatial-data-analysis/code/../data/RawTransactionData/04012016_04302016.csv',
 '/Users/tfiez/GitHub/spatial-data-analysis/code/../data/RawTransactionData/04012017_04302017.csv',
 '/Users/tfiez/GitHub/spatial-data-analysis/code/../data/RawTransactionData/05012016_05312016.csv',
 '/Users/tfiez/GitHub/spatial-data-analysis/code/../data/RawTransactionData/05012017_05312017.csv',
 '/Users/tfiez/GitHub/spatial-data-analysis/code/../data/RawTransactionData/06012016_06302016.csv',
 '/Users/tfiez/GitHub/spatial-data-analysis/code/../data/RawTransactionData/06012017_06302017.csv',


In [7]:
filenames = [fi for fi in filenames if '2016' in fi]

In [8]:
filenames

['/Users/tfiez/GitHub/spatial-data-analysis/code/../data/RawTransactionData/03012016_03312016.csv',
 '/Users/tfiez/GitHub/spatial-data-analysis/code/../data/RawTransactionData/04012016_04302016.csv',
 '/Users/tfiez/GitHub/spatial-data-analysis/code/../data/RawTransactionData/05012016_05312016.csv',
 '/Users/tfiez/GitHub/spatial-data-analysis/code/../data/RawTransactionData/06012016_06302016.csv',
 '/Users/tfiez/GitHub/spatial-data-analysis/code/../data/RawTransactionData/07012016_07312016.csv']

In [9]:
months = [3,4,5,6,7]
years = [2016,2016,2016,2016,2016]

In [ ]:
subareas = ['First Hill', 'Denny Triangle', 'Commercial Core', 'Belltown', 'Pike-Pine']

In [10]:
subareas = ['Belltown']

In [ ]:
load_sdot_utils.create_loads(subareas, months, years, filenames, data_path, verbose=True)

Getting loads for month 3 and year 2016 for subarea Belltown


In [ ]:
start_hour = 8
end_hour = 20
minute_interval = 60
file_paths = [belltown_path]

months = [3,4,5,6,7]
years = [2016]

In [ ]:
load_sdot_utils.aggregate_loads(start_hour, end_hour, minute_interval, months, years, file_paths)

In [ ]:
for path in file_paths:

    # Creating new directory to write new files to if it does not exist.
    subarea_name = path.split('/')[-1].split('_')[0]
    subdir = path + '/..'
    new_dir = subdir + '/' + subarea_name

    if minute_interval == 1:
        dir_addon = '_Minute'
    elif minute_interval == 60:
        dir_addon = '_Hour'
    else:
        dir_addon = '_' + str(minute_interval) + 'Minute' 

    new_dir += dir_addon

    if not os.path.exists(new_dir):
        os.makedirs(new_dir)

    park_data = defaultdict(list)

    month_map = {'Jan':1, 'Feb':2, 'Mar':3, 'Apr':4, 'May':5, 'Jun':6, 'Jul':7, 
                 'Aug':8, 'Sep':9, 'Oct':10, 'Nov':11, 'Dec':12}

    for fi in glob.glob(path + '/*.csv'):
        split = fi.split('/')[-1].split('-')
        key = split[0]
        date = split[1]
        date = date[:4] + '-' + date[4:]
        year, month = int(date.split('-')[0]), date.split('-')[1]
        
        if key == '13297' and month == 'Mar':
            print year, month, fi
            print 'breaking'
            break
        
        if year not in years and month not in months:
            continue

        data = pd.read_csv(fi)
        cols = data.columns
        interval_data = []

        # Aggregate from 1 minute intervals to specified interval.
        for i in xrange(0, 1440, minute_interval):
            interval = data.loc[i:i+minute_interval-1]
            interval_avg = interval.values.mean(axis=0)
            interval_data.append(interval_avg)

        interval_data = np.vstack((interval_data))
        interval_data = interval_data[start_hour*(60/minute_interval):end_hour*(60/minute_interval)]
        interval_data = interval_data.T
        interval_data = interval_data.flatten()

        times = [(hour, minute) for hour in xrange(start_hour, end_hour) for minute in xrange(0, 60, minute_interval)]

        index = [datetime.datetime(year, month_map[month], int(day), hour, minute, 0) for day in cols.tolist() for hour, minute in times]

        new_df = pd.DataFrame(interval_data, index=index, columns=['Load'])
        new_df.index.name = 'Datetime'
        park_data[int(key)].append(new_df.copy())
    break
    for key in park_data:
        park_data[key] = pd.concat(park_data[key], axis=0)
        park_data[key] = park_data[key].sort_index(axis=0)
        park_data[key] = park_data[key].reset_index()
        park_data[key].to_csv(os.path.join(new_dir, str(key)+'.csv'), sep=',', index=False, header=False)

In [ ]:
fi

In [ ]:
check = pd.read_csv(fi)

In [ ]:
check.loc[check['1'] != 0]